In [48]:
props=list()
propDict={'Price':'','Location':'','Bedrooms':'','Area':''}
lastCopy=propDict.copy()
countDec=0
tempCount=[]
tempBath=[]
tempBath2=[]
#finList=[]

import requests
import re
from bs4 import BeautifulSoup

for i in range(0,3000,120):
    
    res=requests.get(f'https://sfbay.craigslist.org/search/apa?s={i}')
    soup=BeautifulSoup(res.text,'html.parser')
    finList=[]
    tempPrime=[]
    
    for count,tempStr in enumerate(soup.select('.result-info')):
        temp=[]
        temp=re.findall('[\d]+ft',str(tempStr))
        if temp!=[]:
            propDict.update({'Area':int(temp[0].replace('ft',''))})
        else:  
            propDict['Area']='None'
            
############################################################################## 

        tempPrime=[]
    
        matPatrn='[a-zA-Z]*\s*studio\s*[a-zA-Z]*'
        tempPrime=re.findall(matPatrn,str(tempStr).lower(),flags=re.IGNORECASE)
        
        removWordsPrime=['estudio','estudio de','from studio']
        
        if tempPrime!=[] and tempPrime not in removWordsPrime:
            propDict['Is Studio']=tempPrime
        else:
            propDict['Is Studio']='NA'
            
############################################################################
        matPatrn='[a-zA-Z0-9]*\s*patio\s*[a-zA-Z0-9]*'
    
        takeWords={'private':'Has a Private patio','Private':'Has a Private patio',
                   'Large':'Has a Large patio','enclosed':'Has a Enclosed Patio',
                   'large':'Has a Large Patio',
                   'Spacious':'Has a Spacious patio'}
        
        tempPrime=[]
        tempPrime=re.findall(matPatrn,str(tempStr).lower(),flags=re.IGNORECASE)
        
        if tempPrime!=[]:
            flag=0
            for j in tempPrime:
                tempLst=[]
                tempLst=j.split(' ')
                for idx in tempLst:
                    idx=idx.replace(' ','')
                    for idx2 in takeWords.keys():
                        if idx2==idx:
                            propDict['Patio']=(takeWords[idx2],count,i)
                            flag=1
                            break
                    break      
            if flag!=1:
                propDict['Patio']='Has Patio'
        else:
            propDict['Patio']='NA'
            
################################################################################ 

        initMatPatrn='WE ♥ PETS|pets are [a-z,A-Z]+|Pets are [a-z,A-Z]+|Pets [a-z,A-Z]+|pets [a-z,A-Z]+|Pet [a-z,A-Z]+|pet [a-z,A-Z]+'

        temp4=[]
        temp4=re.findall(initMatPatrn,str(tempStr))
        
        if temp4!=[]:
            if temp4[0]=='peting' or temp4[0]=='petaluma' :
                propDict['Pet friendly']='NA'
            else:
                if 'Pet Deposit,' not in temp4:
                    propDict['Pet friendly']='Yes' 
        else:
            propDict['Pet friendly']='NA'

################################################################################

        temp1=[]
    
        temp1=re.findall('"result-price">\S*(\$[\d]+)',str(tempStr))
        if temp1!=[]:
            temp2=temp1[0].strip('$')
            propDict.update({'Price':int(temp2)})
        else:  
            propDict['Price']='None'
            
################################################################################## 

        temp2=[]
        temp2=re.findall('<span class="result-hood">([\a-z]+)</span>\n<span class="result-tags">',str(tempStr))
        if temp2!=[]:
            temp3=temp2[0].strip('()')
            propDict.update({'Location':temp3})
        else:  
            propDict['Location']='None'
            
##################################################################################  

        temp3=[]
        matPatt='[\d]br |[\d]BR |[\d]Bedroom |[\d]bedroom |[\d] BR|[\d] br|[\d] bedroom|[\d] Bedroom'
        temp3=re.findall(matPatt,str(tempStr))
        
        if temp3!=[]:
            temp4=temp3[0].replace('br','').replace(' ','')
            if len(temp4)>2 and len(temp4)<8:
                temp4=temp3[0].replace('BR','').replace(' ','')
            if len(temp4)>=8:
                temp4=temp3[0].replace('bedroom','').replace(' ','')
            if len(temp4)>=8:
                temp4=temp3[0].replace('Bedroom','').replace(' ','')
            propDict.update({'Bedrooms':int(temp4)})
            tempCount.append(int(temp4))
        else:  
            propDict['Bedrooms']='None'
            
###################################################################################            
            
        temp4=[] 
        initMatPat=\
        '[\d] Baths|[\d] baths|[\d]Bath|[\d] BA|[\d] ba|[\d]BA|[\d]ba|[\d] baths|[\d]baths|[\d]Baths|[\d] bath|[\d]] Bath|[\d]bath|[\d]Bath|[\d]-ba|[\d]-BA|\d\.\dBath|\d\.\d Bath|\d\.\dbath|\d\.\d bath|\d\.\d BA|\d\.\d ba|\d\.\d-BA|\d\.\d-ba|\d\.\dBA|\d\.\dba'
        temp4=re.findall(initMatPat,str(tempStr))
    
        if temp4!=[]:
            temp5=temp4[0]
            matPatrn='Baths|baths|Bath|bath|ba|BA'
            tempPrime=temp5.replace(' ','').replace('-','')
            tempLst=re.sub(matPatrn,'',tempPrime)
            if float(tempLst)!=5.0:
                tempBath.append(float(tempLst))
                propDict.update({'Baths':float(tempLst)})   
        else:
            propDict['Baths']='None'
            
######################################################################################           
            
        initMatPatrn='[a-zA-z0-9]*\s*kitchen\s*[a-zA-Z0-9]*|\W*\s*kitchen\s*\W*|[a-zA-z0-9]*\-kitchen\-[a-zA-Z0-9]*|[a-zA-z0-9]*\W*kitchen\W*[a-zA-Z0-9]*'
        tempkitchen=[]
        removWords=\
        ['and','your','necessities','in','mountain','with','dryer','available','s','w','ette','studiofull', 'd','high','this','milpitas','s','in','','that','wow','Bath','dryer','WITH','','sunnyhuge','|storage|has','jacuzzi','AVAILABLE•','amp; (ingleside','S','space• has','Concept','end','patio:: has',' (ingleside','• Bright living space • Kitchen has it all']
        matPatrn='\|\+|/|\-|\d{2}|1br'  
        matPatrn2='Kitchen|Kitchens|kitchen|kitchens'
        tempCounter=[]
        temp=[]
        temp=re.findall(initMatPatrn,str(tempStr),flags=re.IGNORECASE)
       
        
        if temp!=[]:
            tempCounter.append(count)
            temp1=''
            temp2=''
            temp3=''
            temp1=re.sub(matPatrn,' ',temp[0])
            temp2=re.sub(matPatrn2,'####',temp1,flags=re.IGNORECASE)
            temp3=re.split('####',temp2)
            tempLst=[]
            
            for j in temp3:
                jPrime=''
                jPrime=j.replace(' ','').replace(',','')
                if jPrime not in removWords:
                    tempLst.append(jPrime) 
                    
            finList=[] 
            if 'bbqs'in tempLst or 'chefs' in tempLst or 'stainless' in tempLst:
                finList.append(tempLst)
            else:
                if len(tempLst)!=0:
                    finList.append(' '.join(tempLst))
                else:
                    finList.append('NA')
                    
            propDict['Kitchen attraction']=finList
            
##########################################################################################

        finList=[]
    
        #Taking care of multiple same ads
        if lastCopy!=propDict:
            props.append(propDict.copy())
            lastCopy=propDict.copy()
        else:
            countDec+=1
            continue
    
    
               


In [23]:
res=requests.get(f'https://sfbay.craigslist.org/search/apa?s={1320}')
soup=BeautifulSoup(res.text,'html.parser')

tempStr=soup.select('.result-info')   
print(tempStr[89])

<p class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">favorite this post</span>
</span>
<time class="result-date" datetime="2020-06-01 17:56" title="Mon 01 Jun 05:56:13 PM">Jun  1</time>
<a class="result-title hdrlnk" data-id="7131047815" href="https://sfbay.craigslist.org/sfc/apa/d/san-francisco-3bd-2ba-beautiful-laurel/7131047815.html">3BD/2BA Beautiful Laurel Village Home w/ In-law Unit and Back Yard</a>
<span class="result-meta">
<span class="result-price">$10500</span>
<span class="housing">
                    3br -
                </span>
<span class="result-hood"> (laurel hts / presidio)</span>
<span class="result-tags">
<span class="pictag">pic</span>
</span>
<span class="banish icon icon-trash" role="button">
<span class="screen-reader-text">hide this posting</span>
</span>
<span aria-hidden="true" class="unbanish icon icon-trash red" role="button"></span>
<a class="restore-link" href="#">
<span class="restore-narrow-text">restor